## CPSC 340 Lecture 22: stochastic gradient descent

This notebook is for the in-class activities. It assumes you have already watched the [associated video](https://www.youtube.com/watch?v=lmqV5Z5HZzc&list=PLWmXHcz_53Q02ZLeAxigki1JZFfCO6M-b&index=22).

<font color='red'>**REMINDER TO START RECORDING**</font>

Also, reminder to enable screen sharing for Participants.

## Pre-class music



1. 夜に駆ける by YOASOBI
2. Heroes by David Bowie

## Admin

- a4 solution posted
- a5 posted
- Final exam scheduling survey: https://ubc.ca1.qualtrics.com/jfe/form/SV_0cGkvmKdr8Aq6SG
- Bug fixed from last class, see https://edstem.org/us/courses/3226/discussion/306056
- Tutorial was missed this morning, rescheduled for 5pm today (right after class)
  - See Canavas for Zoom link

## Video chapters

- kernel trick recap
- stochastic gradient motivation
- SGD vs. GD
- minimizing averages
- per-example gradients
- visualizing SGD: 1 parameter
- decreasing step sizes
- stochastic average gradient
- visualizing SGD: 2 parameters
- summary

## True/False questions

1. For stochastic gradient to work, we need to increase the step size as the optimization proceeds.
2. Stochastic gradient can be used for training many models, including linear regression and logistic regression.
3. An iteration of stochastic gradient might cause the loss might go up, even for a very small learning rate.
4. It is reasonable to check whether the loss went up or down after each iteration of stochastic gradient.

## More T/F questions

These questions pertain to the idea of batch size (aka minibatch size) and epochs:

1. Increasing the batch size results in slower, but better, iterations.
2. If you double the number of epochs and double the batch size, then nothing has changed.
3. Increasing the batch size makes one epoch slower.
4. One epoch of stochastic gradient takes about the same amount of time as one iteration of gradient descent.
5. Stochastic gradient with a minibatch size of $n$ is the same thing as gradient descent.


## Old exam questions

#### 2016W2 final

Name an advantage and a disadvantage of using a smaller minibatch size in stochastic gradient descent.

#### 2018W1 final

![](img/L22/parta.png)

![](img/L22/partb.png)

Note: there is also a part (c), but it's harder than what I would ask about SGD.